In [ ]:
---
title: "Introduction to Quantum Singular Value Transformation"
description: "Or grand unification of quantum algorithms, if you feel a bit grandiose."
aliases:
  - quantum_singular_value_transformation_intro.html
bibliography: references.bib
csl: physical-review-letters.csl
toc: true
image: what_to_use_qc_for.svg
date: '2023-05-15'

format:
  html:
    html-math-method: mathjax
    include-in-header:
      - text: |
          <script>
          window.MathJax = {
            tex: {
              tags: 'ams'
            }
          };
          </script>
---

In [1]:
import numpy as np

# Hamiltonian simulation

We are given three things

1. Block embedding of a Hamiltonian $H$, i.e. $U$ s.t. $H=\left(\langle 0|\otimes I \right)U\left(|0\rangle\otimes I\right)$.
2. Projectors $\Pi(\phi)$.
3. ??


## Block encodings

Singular value $\sigma$ are positive square roots of eigenvalues $M^\dagger M\ge0$.

To block encode some matrix $M$ we need $||M||\le1$ where $||.||$ is the operator norm or, equivalently, the largest singular value.

In [ ]:
numpy

In [52]:
class BlockEncoding:
    
    def __init__(self, M):
        
        self.num_rows = M.shape[0]
        self.num_cols = M.shape[1]
        self.dim = 2*max(self.num_rows, self.num_cols)
        
        self.diagonal_block = self.pad_matrix_to_square(M)
        
        U, S, WH = np.linalg.svd(self.diagonal_block)
        
        self.U = U
        self.S = S
        self.WH = WH
        
        self.off_diagonal_block = self.make_off_diagonal_block(self.U, self.S, self.WH)
        
        self.unitary = self.unitary_from_blocks(
            self.diagonal_block, 
            self.off_diagonal_block)
                    
    @staticmethod
    def pad_matrix_to_square(M):
        """Pads a matrix with zeros to make it square."""

        n_rows, n_cols = M.shape
        n_max = max(n_rows, n_cols)
        S = np.zeros((n_max, n_max))
        S[:n_rows,:n_cols] = M
        return S
    
    @staticmethod
    def make_off_diagonal_block(u, s, wh):
        assert np.all(s<=1), f'All singular values {s} must be less than 1.'
        return u @ np.diag(np.sqrt(1-s**2)) @ wh
    
    @staticmethod
    def unitary_from_blocks(A, B):
        n = A.shape[0]
        U = np.zeros((2*n, 2*n))
        U[:n, :n] = A
        U[:n,n:2*n] = B
        U[n:2*n, :n] = B
        U[n:2*n, n:2*n] = -A
        
        assert np.allclose(U @ U.conj().T, np.eye(2*n)), 'Ops, block encoding is not unitary.'
        return U